## Json Parsing And Processing

In [1]:
import json
import os

os.makedirs("data/json_files", exist_ok=True)

In [2]:
# Sample nested JSON data
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

In [3]:
with open('data/json_files/company_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)

In [6]:
# Save JSON Lines format
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open('data/json_files/events.jsonl', 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item) + '\n')


## Json Processing Strategies

In [ ]:
from langchain_community.document_loaders import JSONLoader
import json


In [13]:
# Method1: JsonLoader with jq_schema

print(f" JSONLoader - Extract specific fields")

employee_loader = JSONLoader(
    file_path='data/json_files/company_data.json',
    jq_schema='.employees[]', # jq query to extract each employee
    text_content=False # Get full JSON objects
)

employee_docs = employee_loader.load()

print(f"Loaded {len(employee_docs)} employee documents")
print(f"First Employee: {employee_docs[0].page_content[:200]}...")


 JSONLoader - Extract specific fields
Loaded 2 employee documents
First Employee: {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status"...


In [14]:
# Method 2: Custom JSON processing for complex structures
from typing import List
from langchain_core.documents import Document

print("\n Custom JSON Processing")

def process_json_intelligently(filepath: str) -> List[Document]:
    """Process JSON with intelligent flattening and context preservation"""

    with open(filepath, 'r') as f:
        data = json.load(f)
    
    documents = []

    # Strategy 1: Create documents for each employee with full context

    for emp in data.get('employees',[]):
        content = f"""Employee Profile:
        Name: {emp['name']}
        Role: {emp['role']}
        Skills: {', '.join(emp['skills'])}
        Projects: """

        for proj in emp.get('projects', []):
            content += f"\n- {proj['name']} (Status: {proj['status']})"

        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )

        documents.append(doc)

    return documents






 Custom JSON Processing


In [15]:
process_json_intelligently('data/json_files/company_data.json')

[Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 1, 'employee_name': 'John Doe', 'role': 'Software Engineer'}, page_content='Employee Profile:\n        Name: John Doe\n        Role: Software Engineer\n        Skills: Python, JavaScript, React\n        Projects: \n- RAG System (Status: In Progress)\n- Data Pipeline (Status: Completed)'),
 Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 2, 'employee_name': 'Jane Smith', 'role': 'Data Scientist'}, page_content='Employee Profile:\n        Name: Jane Smith\n        Role: Data Scientist\n        Skills: Python, Machine Learning, SQL\n        Projects: \n- ML Model (Status: In Progress)\n- Analytics Dashboard (Status: Planning)')]